# Training classifiers to predict the presence of mutations in specific genes using bulk AML RNA-seq data, specifically Beat AML waves 1-4. These signatures will later be applied to the Van Galen single-cell RNA-seq data.

Andrew Ashford, Pathways + Omics Group, Oregon Health & Science University - 11/14/2024

This Jupyter Notebook will read in the previously-integrated AnnData object in which I used Scanorama to integrate both the Beat AML data and the Van Galen single-cell RNA-seq data. I used 200 lower-dimensional features for this integration. I will then apply the classifiers to the single-cell data and predict mutation status for several genes of interest that are associated with AML. Of note, all features were used that were shared between the Beat AML data and the Van Galen data (total = 22681 features). We will additionally apply the signatures to the single-cell data and then save the results and use them for downstream mutation prediction validation and analysis.


#### Import modules

In [1]:
# Import modules
import pandas as pd
import numpy as np
import scanpy as sc
from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score



KeyboardInterrupt



#### Specify directory and file locations and load in integrated data object

In [ ]:
# Specify the Van Galen directory location
van_galen_data_dir = '/home/groups/precepts/ashforda/scOPE_github_stuff/data/testing/vanGalen_all_h5ad/'

# Specify the Scanorama-integrated h5ad file name
#integrated_data_file = 'Scanorama-integrated_combined_Beat_AML_Van_Galen_AML_adata_all_shared_features_22681.h5ad'
#integrated_data_file = 'Scanorama-integrated_combined_Beat_AML_Van_Galen_AML_adata_all_shared_features_22681_imputed_scRNA.h5ad'
integrated_data_file = 'Scanorama-integrated_combined_Beat_AML_Van_Galen_AML_adata_8000_scRNA_features_imputed_scRNA.h5ad'


In [ ]:
# Load the AnnData object
integrated_data = sc.read_h5ad(van_galen_data_dir + integrated_data_file)


In [ ]:
# Sanity check
print(integrated_data)
print(integrated_data.obs)
print(integrated_data.X)
print(integrated_data.X.max())
print(integrated_data.X.min())


In [ ]:
# UMAP visualization of integrated data as a check
sc.pp.neighbors(integrated_data)
sc.tl.umap(integrated_data)


In [ ]:
# Look at UMAP by cell type
sc.pl.umap(integrated_data, color=['CellType'])


In [ ]:
# Look at UMAP by modality type
sc.pl.umap(integrated_data, color=['batch'])


#### Specify file locations and load in the Beat AML mutation data

In [ ]:
# Load the bulk RNA-seq mutation data
mut_data = pd.read_csv('/home/groups/precepts/ashforda/scOPE_github_stuff/data/training/BeatAML/publicly_available_and_raw_counts/beataml_wes_wv1to4_mutations_dbgap.txt', sep='\t')


In [ ]:
# Make an object that spits the bulk data from the rest
# Should be 707 samples.
bulk_data = integrated_data[integrated_data.obs['batch'] == 'bulk']

print(bulk_data)
print(bulk_data.obs_names)


In [ ]:
# Need to add a different suffix to the sample names to match them with the mutation data
bulk_data.obs_names = bulk_data.obs_names.str.replace('R-bulk', 'D')


In [ ]:
# Modify the sample IDs to match between RNA and mutation data
bulk_data.obs['dbgap_sample_id'] = bulk_data.obs.index.str.replace('R-1', 'D')


In [ ]:
# Sanity check the mutation data - make sure the "dbgap_sample_id" column matches the format of the above
# samples minus their "R-bulk" string replaced with "D"
print(mut_data)


In [ ]:
# Ensure the mutation labels are binary
mutations = mut_data['symbol'].unique()

print(mutations)
print(len(mutations))
print(set(mutations))
print(len(set(mutations)))


In [ ]:
# Remove duplicate sample IDs in mutation data (if any)
mut_data = mut_data.drop_duplicates(subset=['dbgap_sample_id', 'symbol'])

# Create a binary matrix for mutation presence
mut_data_binary = pd.crosstab(mut_data['dbgap_sample_id'], mut_data['symbol'])


In [ ]:
print(bulk_data.obs)


In [ ]:
print(mut_data_binary)


In [ ]:
# Align mutation data with bulk RNA-seq data
bulk_sample_ids = bulk_data.obs['dbgap_sample_id']
mut_data_agg = mut_data_binary.reindex(bulk_sample_ids, fill_value=0)

print(mut_data_agg)


In [ ]:
print(bulk_data.X)


#### Train classifiers

In [ ]:
# Just need to set the "X_pca_scaled variable", the variable name doesn't make a ton of sense but it's what
# the code downstream is using. It's just the values for the bulk RNA-seq data
X_pca_scaled = bulk_data.X


In [ ]:
# Initialize models
ridge_model = RidgeClassifierCV()
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
svm_model = SVC(kernel='linear', probability=True, random_state=42)


In [ ]:
# Function to train and evaluate model with Stratified K-Folds
def train_evaluate_model(X, y, model, model_name, cv_splits, mut_name, auc_dict):
    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=42)
    #scores = cross_val_score(model, X, y, cv=skf, scoring='accuracy')
    scores = cross_val_score(model, X, y, cv=skf, scoring='roc_auc')
    #print(f"{model_name} - Cross-validation accuracy: {np.mean(scores)}")
    
    model_rstrip = model_name.rstrip(' ')
    auc_dict[mut_name][model_rstrip].append(np.mean(scores))
    
    print(f"{model_name} - Cross-validation ROC AUC: {np.mean(scores)}")


In [ ]:
# User-defined minimum number of positive samples required
MIN_POSITIVE_SAMPLES = 10
DEFAULT_CV_SPLITS = 10


In [ ]:
# Track mutations with sufficient positive samples
valid_mutations = []

# Keep the ROC AUC scores for each mutation in a dictionary
roc_auc_dict = {}

# Train and evaluate models
for mutation in mutations:
    #print(mutation)
    if mutation in mut_data_agg.columns:
        #print('mutation is in mut_data_agg.columns')
        
        y_mutation = mut_data_agg[mutation].astype(int)
        #print(y_mutation)
        
        # Check the distribution of the labels to ensure it makes sense
        positive_samples = np.sum(y_mutation)
        negative_samples = len(y_mutation) - positive_samples
        
        if positive_samples < MIN_POSITIVE_SAMPLES:
            continue
        
        # Dynamically set the number of CV splits
        cv_splits = min(DEFAULT_CV_SPLITS, positive_samples, negative_samples)
        
        if cv_splits < 2:
            continue
            
        if mutation not in roc_auc_dict.keys():
            roc_auc_dict[mutation] = {'Ridge Classifier': [], 'Random Forest Classifier': [], 'SVM Classifier': []}
        
        #print(roc_auc_dict)
        
        print(f"Training models for mutation: {mutation}")
        
        # Sample equal number of negative samples
        positive_indices = np.where(y_mutation == 1)[0]
        negative_indices = np.where(y_mutation == 0)[0]
        
        negative_indices_sampled = resample(negative_indices, replace=False, n_samples=positive_samples, random_state=42)
        
        # Combine positive and negative samples
        combined_indices = np.concatenate([positive_indices, negative_indices_sampled])
        X_pca_scaled_sampled = X_pca_scaled[combined_indices]
        y_mutation_sampled = y_mutation.iloc[combined_indices]
        
        train_evaluate_model(X_pca_scaled_sampled, y_mutation_sampled, ridge_model, "Ridge Classifier ", cv_splits, mutation, roc_auc_dict)
        train_evaluate_model(X_pca_scaled_sampled, y_mutation_sampled, rf_model, "Random Forest Classifier ", cv_splits, mutation, roc_auc_dict)
        train_evaluate_model(X_pca_scaled_sampled, y_mutation_sampled, svm_model, "SVM Classifier ", cv_splits, mutation, roc_auc_dict)
        
        valid_mutations.append(mutation)
        


#### Validate some of the results

In [ ]:
# AUC score dict sanity check
print(roc_auc_dict)


In [ ]:
performance_dict = roc_auc_dict


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming performance_dict is your dictionary with the classifier performances

# Extract unique classifier names
classifiers = list(next(iter(performance_dict.values())).keys())

# Prepare data for plotting
genes = list(performance_dict.keys())
x = np.arange(len(genes))  # Gene indices
width = 0.2  # Width of the bars

# Initialize figure and axis
plt.figure(figsize=(20, 10))

# Create a bar for each classifier
for i, clf_name in enumerate(classifiers):
    means = [np.mean(performance_dict[gene][clf_name]) for gene in genes]
    stds = [np.std(performance_dict[gene][clf_name]) for gene in genes]
    plt.bar(x + i*width, means, width, yerr=stds, label=clf_name, capsize=5)

# Customize the plot
plt.xticks(x + width, genes, rotation=90)
plt.xlabel('Genes')
plt.ylabel('Performance (AUC)')
plt.title('Classifier Performance Across Genes')
plt.ylim(0, 1)
plt.legend()
plt.tight_layout()

# Show the plot
plt.show()



#### Apply trained mutation classifiers to the single-cell data

In [ ]:
# Again, this variable name doesn't make a ton of sense, but it's just scRNA AnnData .X values
#scRNA_data = integrated_data[integrated_data.obs['dataset'] == '0']
scRNA_data = integrated_data[integrated_data.obs['batch'] == 'scRNA']

# NOTE: Using .X instead of .obsm['X_pca']
#X_scRNA_pca = scRNA_data.obsm['X_pca']
X_scRNA_pca = scRNA_data.X
#X_scRNA_pca_scaled = scaler.transform(X_scRNA_pca)
X_scRNA_pca_scaled = X_scRNA_pca


In [ ]:
# Apply trained models to scRNA-seq data
print('Beginning to apply trained models to scRNA-seq data..')

# Dictionary to store prediction results
predictions = {}

print(valid_mutations)

# Apply predictions only for valid mutations
for mutation in valid_mutations:
    
    print(f'Predicting mutation status in single-cell data for mutation: {mutation}..')
    y_mutation = mut_data_agg[mutation].astype(int)
    
    # Train the final models
    ridge_model.fit(X_pca_scaled, y_mutation)
    rf_model.fit(X_pca_scaled, y_mutation)
    svm_model.fit(X_pca_scaled, y_mutation)
    
    # Predict using scRNA-seq data
    scRNA_data.obs[f'ridge_preds_{mutation}'] = ridge_model.predict(X_scRNA_pca_scaled)
    scRNA_data.obs[f'rf_preds_{mutation}'] = rf_model.predict(X_scRNA_pca_scaled)
    scRNA_data.obs[f'svm_preds_{mutation}'] = svm_model.predict(X_scRNA_pca_scaled)
    
    # Save predictions to dictionary
    predictions[f'ridge_preds_{mutation}'] = scRNA_data.obs[f'ridge_preds_{mutation}']
    predictions[f'rf_preds_{mutation}'] = scRNA_data.obs[f'rf_preds_{mutation}']
    predictions[f'svm_preds_{mutation}'] = scRNA_data.obs[f'svm_preds_{mutation}']

print("Models trained and predictions applied to scRNA-seq data.")


In [ ]:
# Again, there are variables specified above for each of the datasets that have the variable names:
# X_pca_scaled for the training features of the bulk data
# X_scRNA_pca_scaled which are the features to use for the scRNA-seq data

# If those were'nt specified above you can do so here:
#X_pca = bulk_data.X
#X_scRNA_pca_scaled = scRNA_data.X 

# These variables are used by the model to use the bulk features to train models, then apply the single-cell
# features to get single-cell mutation classification probabilty predictions

# Alternatively, if you wanted to train and predict using just the RNA features alone unintegrated, you could 
# set X_pca_scaled to the shared RNA features of the bulk data and X_scRNA_pca_scaled to the shared RNA features
# of the scRNA data.


In [ ]:
# Can apply different settings to the ridge regression classifier to account for scRNA dropout issues
'''
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.calibration import CalibratedClassifierCV

# Define a custom range of alpha values
alphas = np.logspace(-6, 6, 13)

# Initialize the Ridge Classifier with the custom alpha values and balanced class weights
ridge_classifier = RidgeClassifierCV(alphas=alphas, class_weight='balanced')

# Wrap with CalibratedClassifierCV
calibrated_ridge = CalibratedClassifierCV(ridge_classifier, method='sigmoid')

# Fit the model
calibrated_ridge.fit(X_train, y_train)
'''


In [ ]:
# Alternatively, I used this code previously to get the PROBABILITY of having a mutation from the classifier.
# This code does both the training and application to the single-cell data
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Modules for new models
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Original models
'''
# Initialize your models
#ridge_model = RidgeClassifier()  # Note: RidgeClassifier does not have predict_proba
#rf_model = RandomForestClassifier()
#svm_model = SVC(probability=True)  # Make sure SVM is configured to output probabilities
'''
# Model training with hyperparameter tuning and calibration
ridge_model = CalibratedClassifierCV(RidgeClassifierCV(), method='sigmoid')
rf_model = GridSearchCV(RandomForestClassifier(), param_grid={'n_estimators': [100, 200], 'max_depth': [5, 10]}, cv=5)
svm_model = CalibratedClassifierCV(SVC(probability=True), method='sigmoid')
#gbm_model = GridSearchCV(GradientBoostingClassifier(), param_grid={'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1]}, cv=5)

# Apply trained models to scRNA-seq data
print('Beginning to apply trained models to scRNA-seq data..')

# Dictionary to store prediction results
predictions = {}

print(valid_mutations)

# Apply predictions only for valid mutations
for mutation in valid_mutations:
    print(f'Predicting mutation status in single-cell data for mutation: {mutation}..')
    y_mutation = mut_data_agg[mutation].astype(int)
    
    # Train the final models on bulk RNA-seq data
    ridge_model.fit(X_pca_scaled, y_mutation)
    rf_model.fit(X_pca_scaled, y_mutation)
    svm_model.fit(X_pca_scaled, y_mutation)
    #gbm_model.fit(X_pca_scaled, y_mutation)
    
    '''
    # Predict using scRNA-seq data
    # RidgeClassifier does not have predict_proba, using decision_function instead
    ridge_probs = ridge_model.decision_function(X_scRNA_pca_scaled)
    rf_probs = rf_model.predict_proba(X_scRNA_pca_scaled)[:, 1]
    svm_probs = svm_model.predict_proba(X_scRNA_pca_scaled)[:, 1]
    '''
    # Predict using scRNA-seq data
    ridge_probs = ridge_model.predict_proba(X_scRNA_pca_scaled)[:, 1]
    rf_probs = rf_model.predict_proba(X_scRNA_pca_scaled)[:, 1]
    svm_probs = svm_model.predict_proba(X_scRNA_pca_scaled)[:, 1]
    #gbm_probs = gbm_model.predict_proba(X_scRNA_pca_scaled)[:, 1]
    
    # Combine predictions using an ensemble method (e.g., average)
    # Idk if this is a common practice. Commented it out for now.
    #ensemble_probs = (ridge_probs + rf_probs + svm_probs + gbm_probs) / 4
    
    scRNA_data.obs[f'ridge_preds_{mutation}'] = ridge_probs
    scRNA_data.obs[f'rf_preds_{mutation}'] = rf_probs
    scRNA_data.obs[f'svm_preds_{mutation}'] = svm_probs
    # New preds
    #scRNA_data.obs[f'gbm_preds_{mutation}'] = gbm_probs
    #scRNA_data.obs[f'ensemble_preds_{mutation}'] = ensemble_probs
    
    # Save predictions to dictionary
    predictions[f'ridge_preds_{mutation}'] = ridge_probs
    predictions[f'rf_preds_{mutation}'] = rf_probs
    predictions[f'svm_preds_{mutation}'] = svm_probs
    # New preds
    #predictions[f'gbm_preds_{mutation}'] = gbm_probs
    #predictions[f'ensemble_preds_{mutation}'] = ensemble_probs

    '''
    # Additionally, save the predicted probabilities for the bulk RNA-seq data
    ridge_bulk_probs = ridge_model.decision_function(X_pca_scaled)
    rf_bulk_probs = rf_model.predict_proba(X_pca_scaled)[:, 1]
    svm_bulk_probs = svm_model.predict_proba(X_pca_scaled)[:, 1]
    '''
    ridge_bulk_probs = ridge_model.predict_proba(X_pca_scaled)[:, 1]
    rf_bulk_probs = rf_model.predict_proba(X_pca_scaled)[:, 1]
    svm_bulk_probs = svm_model.predict_proba(X_pca_scaled)[:, 1]
    
    predictions[f'ridge_bulk_probs_{mutation}'] = ridge_bulk_probs
    predictions[f'rf_bulk_probs_{mutation}'] = rf_bulk_probs
    predictions[f'svm_bulk_probs_{mutation}'] = svm_bulk_probs

print("Models trained and predictions applied to scRNA-seq data.")


In [ ]:
# The scRNA AnnData object now how the prediction scores within the .obs variable named after the classifier
# used, then named after the gene it's predicting mutations in
print(scRNA_data)
print(scRNA_data.obs['ridge_preds_DNMT3A'])


In [ ]:
# Save the scRNA predictions object for future analysis and validation
scRNA_data.write(van_galen_data_dir + 'Scanorama-integrated_Beat_AML_Van_Galen_AML_adata_8000_scRNA_features_imputed_scRNA_single-cell_predictions.h5ad')
